In [2]:
import numpy as np
import scipy.io as scio
import scipy
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
data_path = "DATA/DATA/FLUIDS/CYLINDER_ALL.mat"
data = scio.loadmat(data_path)
cc_path = "CODE/CH02_FLUIDS/CCcool.mat"
cc = scio.loadmat(cc_path)
cc = cc['CC']
cmap = mcolors.ListedColormap(cc)
%matplotlib qt5
Uall = data['UALL']
Vall = data['VALL']
VortAll = data['VORTALL']
m = data['m'][0][0]
n = data['n'][0][0]
nx = data['nx'][0][0]
ny = data['ny'][0][0]
Uall.shape, Vall.shape, VortAll.shape, m, n, nx, ny

((89351, 151), (89351, 151), (89351, 151), 199, 449, 199, 449)

In [26]:
X = VortAll.T
Y = np.vstack((X, X))
Y.shape

UX = Y[:, 0:Y.shape[1]-1]
UY = Y[:, 1:Y.shape[1]]

UX2 = VortAll[:, 0:VortAll.shape[1]-1]
UY2 = VortAll[:, 1:VortAll.shape[1]]

In [27]:
def DMD_class(X, Y):
# DMD经典算法
# 输入变量X, Y分别为时空矩阵Uxt的1~N-1列以及2~N列
# 输出变量Dd为经过DMD分解后排序过的特征值
# 输出变量b为模态对应的初始结果，与模态相乘后可得初始结果Ux1
# 输出变量Time_DMD为分解后的时间序列（已排序）
# 输出变量Phi为DMD分解后的模态结果（已排序）
# 输出变量Energy为DMD分解后的模态能力值（已排序）
    N = X.shape[1]
    # Step1 对X进行svd分解
    U, S, VT = scipy.linalg.svd(X, full_matrices=False)
    Sd = np.diag(S) 
    r = np.sum(S > 1e-5)
    U = U[:, 0:r]
    Smatrix = Sd[0:r, 0:r]
    V = VT.T[:, 0:r]

    # Step2 求解转换矩阵A
    A = U.T @ Y @ V @ scipy.linalg.inv(Smatrix)
    # Step3 求解矩阵A的特征向量及特征值
    D, Om = scipy.linalg.eig(A)
    Dd = np.diag(D)
    # Step4 求DMD模态
    Phi = Y @ V @ scipy.linalg.inv(Smatrix) @ Om
    # Step5 计算模态对应的初始值b
    b, residuals, rank, singular_values = np.linalg.lstsq(Phi, X[:, 0], rcond=None)

    # Step6 模态排序
    Q = np.vander(D, N, increasing=True) # 建立范德蒙矩阵来储存特征值变换
    Time_DMD = b.reshape(len(b), 1) * Q # 获取模态对应的时间系数

    Energy = np.zeros(np.size(Phi, 1))
    for i in range(np.size(Phi, 1)):
        Uxt_DMD_k = np.real(Phi[:, i].reshape((Phi.shape[0], 1)) * Time_DMD[i, :].reshape((1, Time_DMD.shape[1])))
        E_k = np.sum((Uxt_DMD_k ** 2))
        Energy[i] = E_k

    # 对于每个模态的能量进行排序
    Ie = np.argsort(-Energy)
    Energy = Energy[Ie]
    Dd = Dd[Ie]
    b = b[Ie]
    Phi = Phi[:, Ie]
    Time_DMD = Time_DMD[Ie, :]
    
    return Dd,b,Phi,Time_DMD,Energy

In [28]:
# 计算DMD
Dd, b, Phi, Time_DMD, Energy = DMD_class(UX2, UY2) 

In [40]:
plot_durations(np.reshape(np.real(Phi[:, 5]), (ny, nx)).T)

In [43]:
is_ipython = 'qt5' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()
def plot_durations(vort):
    plt.colormap = cmap
    plt.clf()
    
    plt.pcolor(vort, cmap=cmap)
    caxis_range = [int(-np.min(np.abs(np.min(vort)))), int(np.max(vort)), int(np.min(np.abs(np.min(vort)))), int(np.max(vort))]
    plt.clim(caxis_range[0], caxis_range[1])
    # 重置坐标轴
    plt.xticks([1, 50, 100, 150, 200, 250, 300, 350, 400, 449], ['-1', '0', '1', '2', '3', '4', '5', '6', '7', '8'])
    plt.yticks([1, 50, 100, 150, 199], ['2', '1', '0', '-1', '-2'])
    # plt.gcf().set_position([500, 500, 900, 390])
    plt.gca().set_zorder(10)
    plt.ylabel('y', fontname='Arial', fontsize=14)
    plt.xlabel('x', fontname='Arial', fontsize=14)
    plt.axis('equal')
    plt.contour(vort, np.linspace(-np.max(vort), -np.max(vort)/35, 6), colors='k', linestyles='solid', linewidths=1)
    plt.contour(vort, np.linspace(np.max(vort)/35, np.max(vort), 6), colors='k', linestyles='--', linewidths=1)
    theta = np.linspace(0, 2*np.pi, 100)
    x = 49 + 25 * np.sin(theta)
    y = 99 + 25 * np.cos(theta)
    # 绘制填充的圆柱体
    plt.fill(x, y, [.3, .3, .3])
    # 绘制圆柱体的边界
    plt.plot(x, y, 'k', linewidth=1.2)
    plt.pause(0.001)
    if is_ipython:
        display.clear_output(wait=True)
        display.display(plt.gcf())
for i in range(20):
    plot_durations(np.reshape(np.real(Phi[:, i]), (ny, nx)).T)
    # plot_durations(np.reshape(np.imag(Phi[:, i]), (ny, nx)).T)

In [45]:
plt.ioff()

In [29]:
Dd.shape

(150, 150)

In [48]:

# 图1 输出特征根分布
plt.scatter(np.sum(np.real(Dd[::-1]), axis=1),
            np.sum(np.imag(Dd[::-1]), axis=1),
            s=30,
            c=-np.log(Energy[::-1]),
            cmap='viridis',
            alpha=0.7)
plt.colorbar()
plt.axis('equal')
plt.show()

In [49]:
fig2 = plt.figure(2)
# 图2 绘制频率和衰减图
Fs = 1 / 0.05
wa = np.log(np.sum(Dd, axis=1)) * Fs
# 增加一个极小的数防止计算过程出现inf
plt.scatter(np.real(wa),
            np.imag(wa) / (2 * np.pi),
            s=30,
            c=-np.log(Energy),
            cmap='viridis',
            alpha=0.7)
plt.xlabel('AttenuationRatio σ')
plt.ylabel('Frequency w')
plt.colorbar()
plt.xlim(-1, 1)
plt.ylim(-6, 6)
plt.show()

In [50]:
fig3 = plt.figure(3)
# 频率-能量排序
Freq = np.imag(wa) / 2 / np.pi
# 找到大于等于0的频率部分的索引
k = np.where(Freq >= 0)[0]

# 绘制幅度谱
plt.stem(Freq[k], np.log10(Energy[k]), basefmt='b', linefmt='b-', 
         markerfmt='bo')

# 设置基线值为-6
plt.axhline(y=-4, color='r', linestyle='--')

# 显示图形
plt.show()

In [51]:
# 计算UV向量
def Uxyt2Uxt(Uxyt):
    # 把3维矩阵的xyt压缩为xt
    Ny, Nx, Nt = Uxyt.shape # 由于meshgrid定义，xy相反
    Nxy = Ny * Nx
    Uxt = np.reshape(Uxyt, (Nxy, Nt))
    return Uxt

def UV2UxyVxy(UVx, Ny, Nx):
    Ux = UVx[0:Ny*Nx]
    Vx = UVx[Ny*Nx::]
    Uxy = np.reshape(Ux, (Ny, Nx))
    Vxy = np.reshape(Vx, (Ny, Nx))
    return Uxy, Vxy
def curl(Uxy0, Vxy0, X, Y):

    return (np.gradient(Uxy0, axis=0) / 80 + np.gradient(Vxy0, axis=1) / 50)

In [70]:

VortK = np.real(np.outer(Phi[:, 0], Time_DMD[0, :]))
VortK[:, 0].shape

(89351,)

In [64]:
# Uxy0, Vxy0 = UV2UxyVxy(VortAll[:, 0], ny, nx)
plt.pcolor(VortAll[:, 0].reshape((ny, nx)).T)
plt.xticks([1, 50, 100, 150, 200, 250, 300, 350, 400, 449], ['-1', '0', '1', '2', '3', '4', '5', '6', '7', '8'])
plt.yticks([1, 50, 100, 150, 199], ['2', '1', '0', '-1', '-2'])
theta = np.linspace(0, 2*np.pi, 100)
x = 49 + 25 * np.sin(theta)
y = 99 + 25 * np.cos(theta)
# 绘制填充的圆柱体
plt.fill(x, y, [.3, .3, .3])
# 绘制圆柱体的边界
plt.plot(x, y, 'k', linewidth=1.2)
plt.show()

In [75]:
VortK = np.real(np.outer(Phi[:, 0], Time_DMD[0, :]))
vortk = VortK[:, 0].reshape((ny, nx)).T
plt.pcolor(vortk)
plt.xticks([1, 50, 100, 150, 200, 250, 300, 350, 400, 449], ['-1', '0', '1', '2', '3', '4', '5', '6', '7', '8'])
plt.yticks([1, 50, 100, 150, 199], ['2', '1', '0', '-1', '-2'])
theta = np.linspace(0, 2*np.pi, 100)
plt.contour(vortk, np.linspace(-np.max(vortk), -np.max(vortk)/35, 6), colors='k', linestyles='solid', linewidths=1)
plt.contour(vortk, np.linspace(np.max(vortk)/35, np.max(vortk), 6), colors='k', linestyles='--', linewidths=1)
x = 49 + 25 * np.sin(theta)
y = 99 + 25 * np.cos(theta)
# 绘制填充的圆柱体
plt.fill(x, y, [.3, .3, .3])
# 绘制圆柱体的边界
plt.plot(x, y, 'k', linewidth=1.2)
plt.show()

In [77]:
for i in range(100):
    plot_durations(np.reshape(np.real(np.outer(Phi[:, i], Time_DMD[i, :]))[:, i], (ny, nx)).T)
    plt.pause(0.5)

KeyboardInterrupt: 